In [1]:
!pip install pyspark

     |████████████████████████████████| 204.8MB 79kB/s 
     |████████████████████████████████| 204kB 19.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=adb2dc1ddc5206b5f8b46490533def033ae6292499d1f997b83f77182b7a9a1a
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName("Text_Data_Processing").getOrCreate()

In [4]:
import pandas as pd

In [20]:
ms={"Product":['Banana','Carrots','Beans','Orange','Orange','Banana','Carrots','Beans','Orange','Banana','Carrots','Beans'],
    "Amount":[1000,1500,1600,2000,2000,400,1200,1500,4000,2000,2000,2000],
    "Country":['USA','USA','USA','USA','USA','China','China','China','China','Canada','Canada','Mexico']};
gs=pd.DataFrame(ms)

In [32]:
gs.head(5)

,Product,Amount,Country
0,Banana,1000,USA
1,Carrots,1500,USA
2,Beans,1600,USA
3,Orange,2000,USA
4,Orange,2000,USA


In [22]:
data=spark.createDataFrame(gs);
data.show()

+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
| Banana|  1000|    USA|
|Carrots|  1500|    USA|
|  Beans|  1600|    USA|
| Orange|  2000|    USA|
| Orange|  2000|    USA|
| Banana|   400|  China|
|Carrots|  1200|  China|
|  Beans|  1500|  China|
| Orange|  4000|  China|
| Banana|  2000| Canada|
|Carrots|  2000| Canada|
|  Beans|  2000| Mexico|
+-------+------+-------+



In [30]:
data.count()

12

In [23]:
from pyspark.sql.functions import *

In [27]:
data.groupBy('Product').agg(sum('Amount')).show()

+-------+-----------+
|Product|sum(Amount)|
+-------+-----------+
| Orange|       8000|
|  Beans|       5100|
| Banana|       3400|
|Carrots|       4700|
+-------+-----------+



In [29]:
data.groupBy('Product','Country').agg(sum('Amount')).show()

+-------+-------+-----------+
|Product|Country|sum(Amount)|
+-------+-------+-----------+
| Orange|  China|       4000|
| Banana|  China|        400|
|Carrots|    USA|       1500|
|Carrots|  China|       1200|
| Banana|    USA|       1000|
|Carrots| Canada|       2000|
|  Beans|    USA|       1600|
|  Beans| Mexico|       2000|
| Orange|    USA|       4000|
| Banana| Canada|       2000|
|  Beans|  China|       1500|
+-------+-------+-----------+



In [31]:
data.groupBy('Product','Country').agg(sum('Amount')).count()

11

Spark SQL provides pivot() function to rotate the data from one column into multiple columns (transpose row to column).

It is an aggregation where one of the grouping columns values transposed into individual columns with distinct data. 

In [28]:
data.groupBy('Product').pivot('Country').agg(sum('Amount')).show()

+-------+------+-----+------+----+
|Product|Canada|China|Mexico| USA|
+-------+------+-----+------+----+
| Orange|  null| 4000|  null|4000|
|  Beans|  null| 1500|  2000|1600|
| Banana|  2000|  400|  null|1000|
|Carrots|  2000| 1200|  null|1500|
+-------+------+-----+------+----+

